# Multi-Temporal Satellite Band Extraction for Soil Moisture Analysis

This notebook extracts raw bands from Sentinel-1 and Sentinel-2 satellites for soil moisture sampling points using a multi-temporal approach:

## Methodology:
1. **Temporal Strategy**: Extract data from 3 time periods per point:
   - Same date as field measurement
   - One image before (within search window)
   - One image after (within search window)

2. **Sentinel-2**: Extract raw bands (B2, B3, B4, B8, B11, B12)
   - Cloud pixels remain as null/empty values
   - No cloud masking replacement

3. **Sentinel-1**: Extract VV, VH backscatter + incidence angle
   - Include orbit direction (ascending/descending)
   - Include acquisition metadata

4. **Output**: Tidy dataset ready for spectral index calculation and analysis

## 1. Set Up Python and Google Earth Engine

In [31]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import ee
from datetime import datetime, timedelta
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Authenticate and initialize Google Earth Engine
try:
    ee.Initialize()
    print("✓ Google Earth Engine initialized successfully")
except Exception as e:
    print("⚠ Please authenticate Google Earth Engine first:")
    print("Run: ee.Authenticate()")
    ee.Authenticate()
    ee.Initialize()
    print("✓ Google Earth Engine initialized successfully")

# Test connection
test_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').limit(1)
print(f"✓ Connection test: Found {test_collection.size().getInfo()} image(s)")
print(f"✓ Setup complete - Ready for band extraction!")

✓ Google Earth Engine initialized successfully
✓ Connection test: Found 1 image(s)
✓ Setup complete - Ready for band extraction!
✓ Connection test: Found 1 image(s)
✓ Setup complete - Ready for band extraction!


## 2. Load Point Dataset with Timestamps

In [32]:
# Load the shapefile with sampling points
shapefile_path = "soil_moisture_sample_points_7sep.shp"
gdf = gpd.read_file(shapefile_path)

print("=== DATASET OVERVIEW ===")
print(f"Total points: {len(gdf)}")
print(f"Columns: {list(gdf.columns)}")
print(f"Date range: {gdf['Date'].min()} to {gdf['Date'].max()}")
print(f"Unique dates: {gdf['Date'].nunique()}")

# Ensure dates are properly formatted
gdf['Date_dt'] = pd.to_datetime(gdf['Date'])

# Display summary by date
print("\n=== POINTS BY DATE ===")
date_summary = gdf.groupby('Date').size()
for date, count in date_summary.items():
    print(f"  {date.strftime('%Y-%m-%d')}: {count} points")

# Convert to GEE FeatureCollection with essential properties
gee_features = []
for idx, row in gdf.iterrows():
    # Create point geometry
    point = ee.Geometry.Point([row.geometry.x, row.geometry.y])
    
    # Create feature with essential properties
    feature = ee.Feature(point, {
        'point_id': int(row['fid_1']),           # Point identifier
        'point_date': row['Date_dt'].strftime('%Y-%m-%d'),  # Sampling date as string
        'longitude': float(row.geometry.x),      # For reference
        'latitude': float(row.geometry.y)        # For reference
    })
    gee_features.append(feature)

# Create feature collection
points_collection = ee.FeatureCollection(gee_features)
print(f"\n✓ Created GEE FeatureCollection with {points_collection.size().getInfo()} points")

# Show first few points for verification
print("\n=== SAMPLE POINTS ===")
sample_points = points_collection.limit(3).getInfo()['features']
for i, point in enumerate(sample_points):
    props = point['properties']
    coords = point['geometry']['coordinates']
    print(f"  Point {props['point_id']}: ({coords[0]:.5f}, {coords[1]:.5f}) - {props['point_date']}")

=== DATASET OVERVIEW ===
Total points: 23
Columns: ['fid_1', 'N', 'S', 'W', 'E', 'C', 'Date', 'geometry']
Date range: 2025-06-29 00:00:00 to 2025-09-27 00:00:00
Unique dates: 2

=== POINTS BY DATE ===
  2025-06-29: 10 points
  2025-09-27: 13 points

✓ Created GEE FeatureCollection with 23 points

=== SAMPLE POINTS ===

✓ Created GEE FeatureCollection with 23 points

=== SAMPLE POINTS ===
  Point 0: (9.96521, 49.77770) - 2025-06-29
  Point 1: (9.96676, 49.77433) - 2025-06-29
  Point 2: (9.96504, 49.77385) - 2025-06-29
  Point 0: (9.96521, 49.77770) - 2025-06-29
  Point 1: (9.96676, 49.77433) - 2025-06-29
  Point 2: (9.96504, 49.77385) - 2025-06-29


## 3. Configure Search Parameters and AOI

In [33]:
# Configuration parameters
CONFIG = {
    # Temporal search windows
    'max_days_before': 10,  # Maximum days to look for previous image
    'max_days_after': 10,   # Maximum days to look for next image
    
    # Sentinel-2 bands to extract
    's2_bands': ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],  # Blue, Green, Red, NIR, SWIR1, SWIR2
    
    # Sentinel-1 bands to extract
    's1_bands': ['VV', 'VH', 'angle'],  # VV, VH polarizations + incidence angle
    
    # Spatial resolution for sampling
    's2_scale': 10,  # meters
    's1_scale': 10,  # meters
    
    # Area of Interest buffer
    'aoi_buffer': 0.00009,  # degrees (~10m buffer around points)
    
    # Sentinel-1 preprocessing parameters
    's1_speckle_filter_size': 30,  # meters - for speckle filtering
    's1_angle_min': 30.64,  # degrees - minimum incidence angle (border noise removal)
    's1_angle_max': 45.24   # degrees - maximum incidence angle (border noise removal)
}

print("=== EXTRACTION CONFIGURATION ===")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

# Create Area of Interest (AOI) from points with buffer
aoi = points_collection.geometry().bounds().buffer(CONFIG['aoi_buffer'])

print(f"\n✓ AOI created with {CONFIG['aoi_buffer']} degree buffer")

# Define utility functions for date handling
def get_date_range(center_date_str, days_before=0, days_after=0):
    """Get date range for filtering collections"""
    center_date = datetime.strptime(center_date_str, '%Y-%m-%d')
    start_date = center_date - timedelta(days=days_before)
    end_date = center_date + timedelta(days=days_after + 1)  # +1 for inclusive end
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

def get_same_day_range(date_str):
    """Get same day range (date to date+1)"""
    return get_date_range(date_str, 0, 0)

def get_before_range(date_str, max_days):
    """Get range for previous images"""
    center_date = datetime.strptime(date_str, '%Y-%m-%d')
    start_date = center_date - timedelta(days=max_days)
    end_date = center_date  # Up to but not including center date
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

def get_after_range(date_str, max_days):
    """Get range for next images"""
    center_date = datetime.strptime(date_str, '%Y-%m-%d')
    start_date = center_date + timedelta(days=1)  # After center date
    end_date = center_date + timedelta(days=max_days + 1)
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

print("✓ Date utility functions defined")
print("✓ Configuration complete - Ready for image collection setup")

=== EXTRACTION CONFIGURATION ===
  max_days_before: 10
  max_days_after: 10
  s2_bands: ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
  s1_bands: ['VV', 'VH', 'angle']
  s2_scale: 10
  s1_scale: 10
  aoi_buffer: 9e-05
  s1_speckle_filter_size: 30
  s1_angle_min: 30.64
  s1_angle_max: 45.24

✓ AOI created with 9e-05 degree buffer
✓ Date utility functions defined
✓ Configuration complete - Ready for image collection setup


## 4. Build Sentinel-2 Collection (Cloud-Aware)

In [34]:
# Build Sentinel-2 collection
# Get overall date range from points
min_date = gdf['Date_dt'].min() - timedelta(days=CONFIG['max_days_before'])
max_date = gdf['Date_dt'].max() + timedelta(days=CONFIG['max_days_after'])

print("=== SENTINEL-2 COLLECTION SETUP ===")
print(f"Date range: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")

# Create base Sentinel-2 collection (include QA60 for cloud masking)
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                 .filterBounds(aoi)
                 .filterDate(min_date.strftime('%Y-%m-%d'), max_date.strftime('%Y-%m-%d'))
                 .select(CONFIG['s2_bands'] + ['QA60']))

print(f"✓ Base S2 collection: {s2_collection.size().getInfo()} images")

def apply_cloud_mask_s2(image):
    """
    Apply cloud mask to Sentinel-2 image but keep masked pixels as null
    This ensures cloudy pixels return null values when sampled
    """
    # Cloud mask using QA60 band
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Create mask (1 for clear pixels, 0 for clouds)
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    # Apply mask to selected bands and keep only the bands we want
    return image.select(CONFIG['s2_bands']).updateMask(mask).copyProperties(image, ["system:time_start", "system:id"])

# Apply cloud masking to collection
s2_masked = s2_collection.map(apply_cloud_mask_s2)

print("✓ Cloud masking applied - cloudy pixels will return null values")
print("✓ Sentinel-2 collection ready for temporal extraction")

# Test collection availability
print(f"\n=== S2 AVAILABILITY CHECK ===")
# Check a few sample dates
sample_dates = gdf['Date_dt'].unique()[:3]
for sample_date in sample_dates:
    date_str = sample_date.strftime('%Y-%m-%d')
    start_str, end_str = get_same_day_range(date_str)
    same_day = s2_masked.filterDate(start_str, end_str)
    print(f"  {date_str}: {same_day.size().getInfo()} image(s) available")

=== SENTINEL-2 COLLECTION SETUP ===
Date range: 2025-06-19 to 2025-10-07
✓ Base S2 collection: 62 images
✓ Cloud masking applied - cloudy pixels will return null values
✓ Sentinel-2 collection ready for temporal extraction

=== S2 AVAILABILITY CHECK ===
✓ Base S2 collection: 62 images
✓ Cloud masking applied - cloudy pixels will return null values
✓ Sentinel-2 collection ready for temporal extraction

=== S2 AVAILABILITY CHECK ===
  2025-06-29: 1 image(s) available
  2025-06-29: 1 image(s) available
  2025-09-27: 2 image(s) available
  2025-09-27: 2 image(s) available


## 5. Build Sentinel-1 Collection with Orbit/Angle

In [35]:
# Build Sentinel-1 collection with preprocessing
print("=== SENTINEL-1 COLLECTION SETUP WITH PREPROCESSING ===")

# Create base Sentinel-1 collection
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(aoi)
                 .filterDate(min_date.strftime('%Y-%m-%d'), max_date.strftime('%Y-%m-%d'))
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filter(ee.Filter.eq('resolution_meters', 10))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                 .select(CONFIG['s1_bands']))

print(f"✓ Base S1 collection: {s1_collection.size().getInfo()} images")

# Define Sentinel-1 preprocessing functions
def apply_border_noise_removal(image):
    """
    Remove border noise by masking pixels with extreme incidence angles
    This removes edge effects that can contaminate SAR data
    """
    angle = image.select('angle')
    
    # Create mask for valid incidence angles (exclude edges)
    valid_angle_mask = angle.gt(CONFIG['s1_angle_min']).And(angle.lt(CONFIG['s1_angle_max']))
    
    # Apply mask to all bands
    return image.updateMask(valid_angle_mask).copyProperties(image, ['system:time_start', 'system:id'])

def apply_speckle_filter(image):
    """
    Apply median filter to reduce speckle noise in SAR data
    This improves data quality for point-based sampling
    """
    # Apply focal median filter to VV and VH bands
    vv_filtered = image.select('VV').focal_median(CONFIG['s1_speckle_filter_size'], 'square', 'meters')
    vh_filtered = image.select('VH').focal_median(CONFIG['s1_speckle_filter_size'], 'square', 'meters')
    
    # Keep the angle band unchanged and combine with filtered polarizations
    filtered = ee.Image.cat([
        vv_filtered.rename('VV'),
        vh_filtered.rename('VH'),
        image.select('angle')
    ])
    
    return filtered.copyProperties(image, ['system:time_start', 'system:id'])

def add_orbit_properties(image):
    """Add orbit direction and other metadata as properties"""
    # Get orbit properties
    orbit_pass = image.get('orbitProperties_pass')  # ASCENDING or DESCENDING
    relative_orbit = image.get('relativeOrbitNumber_start')
    
    # Add as image properties (will be extracted during sampling)
    return image.set({
        'orbit_direction': orbit_pass,
        'relative_orbit': relative_orbit,
        'acquisition_date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    })

# Apply preprocessing pipeline
print("✓ Applying border noise removal...")
s1_border_cleaned = s1_collection.map(apply_border_noise_removal)

print("✓ Applying speckle filtering...")
s1_speckle_filtered = s1_border_cleaned.map(apply_speckle_filter)

print("✓ Adding orbit metadata...")
s1_with_metadata = s1_speckle_filtered.map(add_orbit_properties)

print(f"✓ Preprocessed S1 collection ready: {s1_with_metadata.size().getInfo()} images")
print(f"  - Border noise removed (angles: {CONFIG['s1_angle_min']:.1f}° - {CONFIG['s1_angle_max']:.1f}°)")
print(f"  - Speckle filtered (kernel size: {CONFIG['s1_speckle_filter_size']}m)")

# Test S1 collection availability
print(f"\n=== S1 AVAILABILITY CHECK ===")
for sample_date in sample_dates:
    date_str = sample_date.strftime('%Y-%m-%d')
    start_str, end_str = get_same_day_range(date_str)
    same_day = s1_with_metadata.filterDate(start_str, end_str)
    
    if same_day.size().getInfo() > 0:
        # Get orbit info for available image
        first_image = same_day.first()
        orbit_info = first_image.getInfo()['properties']
        orbit_dir = orbit_info.get('orbit_direction', 'Unknown')
        print(f"  {date_str}: {same_day.size().getInfo()} image(s) - Orbit: {orbit_dir}")
    else:
        print(f"  {date_str}: No images available")

print("✓ Sentinel-1 collection ready for temporal extraction")

=== SENTINEL-1 COLLECTION SETUP WITH PREPROCESSING ===
✓ Base S1 collection: 69 images
✓ Applying border noise removal...
✓ Applying speckle filtering...
✓ Adding orbit metadata...
✓ Base S1 collection: 69 images
✓ Applying border noise removal...
✓ Applying speckle filtering...
✓ Adding orbit metadata...
✓ Preprocessed S1 collection ready: 69 images
  - Border noise removed (angles: 30.6° - 45.2°)
  - Speckle filtered (kernel size: 30m)

=== S1 AVAILABILITY CHECK ===
✓ Preprocessed S1 collection ready: 69 images
  - Border noise removed (angles: 30.6° - 45.2°)
  - Speckle filtered (kernel size: 30m)

=== S1 AVAILABILITY CHECK ===
  2025-06-29: 1 image(s) - Orbit: Unknown
  2025-06-29: 1 image(s) - Orbit: Unknown
  2025-09-27: 1 image(s) - Orbit: Unknown
✓ Sentinel-1 collection ready for temporal extraction
  2025-09-27: 1 image(s) - Orbit: Unknown
✓ Sentinel-1 collection ready for temporal extraction


## Sentinel-1 Preprocessing Applied

The following preprocessing steps have been applied to improve Sentinel-1 data quality:

### ✅ **Border Noise Removal:**
- **Purpose**: Eliminates edge effects and unreliable pixels at image borders
- **Method**: Masks pixels with incidence angles outside 30.6° - 45.2°
- **Benefit**: Prevents sampling of corrupted edge pixels

### ✅ **Speckle Filtering:**
- **Purpose**: Reduces inherent SAR speckle noise 
- **Method**: Focal median filter with 30m kernel
- **Benefit**: Improves reliability of point-based measurements

### ⏭️ **Skipped Corrections:**
- **Terrain Correction**: Not needed - all points at similar elevation
- **Orbit Filtering**: Maintained diversity for better temporal coverage

This preprocessing significantly improves data quality while preserving the multi-temporal approach needed for your soil moisture analysis.

## 6. Multi-Temporal Image Selection Functions

In [36]:
def get_temporal_images(collection, point_date, max_days_before, max_days_after):
    """
    Get same-day, previous, and next images for a given point date
    Returns dictionary with 'same', 'previous', 'next' keys containing image IDs or None
    """
    results = {'same': None, 'previous': None, 'next': None}
    
    # Same day images
    start_same, end_same = get_same_day_range(point_date)
    same_day_collection = collection.filterDate(start_same, end_same)
    if same_day_collection.size().getInfo() > 0:
        results['same'] = same_day_collection.first().get('system:id')
    
    # Previous images (before point date)
    start_prev, end_prev = get_before_range(point_date, max_days_before)
    prev_collection = collection.filterDate(start_prev, end_prev).sort('system:time_start', False)
    if prev_collection.size().getInfo() > 0:
        results['previous'] = prev_collection.first().get('system:id')  # Most recent before
    
    # Next images (after point date)  
    start_next, end_next = get_after_range(point_date, max_days_after)
    next_collection = collection.filterDate(start_next, end_next).sort('system:time_start')
    if next_collection.size().getInfo() > 0:
        results['next'] = next_collection.first().get('system:id')  # Earliest after
    
    return results

def sample_bands_at_point(image, point_geometry, bands, scale, sensor_type):
    """
    Sample bands at a point location and return properties
    """
    if image is None:
        return None
    
    # Sample the image at point location
    sample = image.sampleRegions(
        collection=ee.FeatureCollection([ee.Feature(point_geometry)]),
        scale=scale,
        projection='EPSG:4326'
    ).first()
    
    # Get sampled values - handle case where sampling returns None
    sample_info = sample.getInfo()
    if sample_info is None or 'properties' not in sample_info:
        print(f"    Warning: No valid data at point for {sensor_type}")
        return None
    
    sampled_data = sample_info['properties']
    
    # Get image metadata
    image_props = image.getInfo()['properties']
    
    # Combine band values with metadata
    result = {}
    
    # Add band values
    for band in bands:
        result[f'{sensor_type.lower()}_{band}'] = sampled_data.get(band, None)
    
    # Add metadata
    result['image_id'] = image_props.get('system:id', '')
    result['acquisition_date'] = image_props.get('acquisition_date', '')
    
    # Add sensor-specific metadata
    if sensor_type == 'S1':
        result['s1_orbit_direction'] = image_props.get('orbit_direction', '')
        result['s1_relative_orbit'] = image_props.get('relative_orbit', '')
    
    return result

print("✓ Multi-temporal extraction functions defined")
print("✓ Ready for band sampling at each point")

✓ Multi-temporal extraction functions defined
✓ Ready for band sampling at each point


## 7. Extract Sentinel-2 Bands (Multi-Temporal)

In [37]:
print("=== EXTRACTING SENTINEL-2 BANDS ===")
print("Note: Cloud pixels will return null values (not masked/replaced)")

# Extract Sentinel-2 data for all points
s2_results = []

# Get point list for processing
points_list = points_collection.getInfo()['features']
total_points = len(points_list)

for i, point_feature in enumerate(points_list):
    point_props = point_feature['properties']
    point_geom = ee.Geometry.Point(point_feature['geometry']['coordinates'])
    point_id = point_props['point_id']
    point_date = point_props['point_date']
    
    print(f"Processing point {point_id} ({i+1}/{total_points}) - Date: {point_date}")
    
    # Get temporal images for this point
    temporal_images = get_temporal_images(
        s2_masked, point_date, 
        CONFIG['max_days_before'], 
        CONFIG['max_days_after']
    )
    
    # Process each temporal slot
    for slot, image_id in temporal_images.items():
        if image_id is not None:
            # Get the actual image
            image_id_str = image_id.getInfo()
            image = s2_masked.filter(ee.Filter.eq('system:id', image_id_str)).first()
            
            # Sample bands at point
            sampled_data = sample_bands_at_point(
                image, point_geom, CONFIG['s2_bands'], CONFIG['s2_scale'], 'S2'
            )
            
            if sampled_data:
                # Add point and slot information
                sampled_data.update({
                    'point_id': point_id,
                    'point_date': point_date,
                    'slot': slot,
                    'sensor': 'S2',
                    'longitude': point_props['longitude'],
                    'latitude': point_props['latitude']
                })
                s2_results.append(sampled_data)
                
                # Show sample extraction info
                cloud_bands = [band for band in CONFIG['s2_bands'] 
                             if sampled_data.get(f's2_{band}') is None]
                if cloud_bands:
                    print(f"  {slot}: {len(cloud_bands)} cloudy bands - {sampled_data['acquisition_date']}")
                else:
                    print(f"  {slot}: Clear data - {sampled_data['acquisition_date']}")
        else:
            print(f"  {slot}: No image available")

print(f"\n✓ Sentinel-2 extraction complete: {len(s2_results)} records")

# Convert to DataFrame for easier handling
s2_df = pd.DataFrame(s2_results)
if len(s2_df) > 0:
    print(f"✓ S2 DataFrame created: {s2_df.shape}")
    print(f"  Columns: {list(s2_df.columns)}")
    
    # Summary of cloud coverage
    s2_band_cols = [col for col in s2_df.columns if col.startswith('s2_')]
    null_counts = s2_df[s2_band_cols].isnull().sum()
    print(f"\n=== CLOUD IMPACT SUMMARY ===")
    for band, null_count in null_counts.items():
        total_extractions = len(s2_df)
        print(f"  {band}: {null_count}/{total_extractions} cloudy ({null_count/total_extractions*100:.1f}%)")
else:
    print("⚠ No Sentinel-2 data extracted")

=== EXTRACTING SENTINEL-2 BANDS ===
Note: Cloud pixels will return null values (not masked/replaced)
Processing point 0 (1/23) - Date: 2025-06-29
Processing point 0 (1/23) - Date: 2025-06-29
  same: Clear data - 
  same: Clear data - 
  next: Clear data - 
Processing point 1 (2/23) - Date: 2025-06-29
  next: Clear data - 
Processing point 1 (2/23) - Date: 2025-06-29
  same: Clear data - 
  same: Clear data - 
  next: Clear data - 
Processing point 2 (3/23) - Date: 2025-06-29
  next: Clear data - 
Processing point 2 (3/23) - Date: 2025-06-29
  same: Clear data - 
  same: Clear data - 
  next: Clear data - 
Processing point 4 (4/23) - Date: 2025-06-29
  next: Clear data - 
Processing point 4 (4/23) - Date: 2025-06-29
  same: Clear data - 
  same: Clear data - 
  next: Clear data - 
Processing point 6 (5/23) - Date: 2025-06-29
  next: Clear data - 
Processing point 6 (5/23) - Date: 2025-06-29
  same: Clear data - 
  same: Clear data - 
  next: Clear data - 
Processing point 7 (6/23) - Dat

## 8. Extract Sentinel-1 Bands with Orbit Metadata

In [38]:
print("=== EXTRACTING SENTINEL-1 BANDS ===")
print("Including VV, VH polarizations, incidence angle, and orbit direction")

# Extract Sentinel-1 data for all points
s1_results = []

for i, point_feature in enumerate(points_list):
    point_props = point_feature['properties']
    point_geom = ee.Geometry.Point(point_feature['geometry']['coordinates'])
    point_id = point_props['point_id']
    point_date = point_props['point_date']
    
    print(f"Processing point {point_id} ({i+1}/{total_points}) - Date: {point_date}")
    
    # Get temporal images for this point
    temporal_images = get_temporal_images(
        s1_with_metadata, point_date, 
        CONFIG['max_days_before'], 
        CONFIG['max_days_after']
    )
    
    # Process each temporal slot
    for slot, image_id in temporal_images.items():
        if image_id is not None:
            # Get the actual image
            image_id_str = image_id.getInfo()
            image = s1_with_metadata.filter(ee.Filter.eq('system:id', image_id_str)).first()
            
            # Sample bands at point
            sampled_data = sample_bands_at_point(
                image, point_geom, CONFIG['s1_bands'], CONFIG['s1_scale'], 'S1'
            )
            
            if sampled_data:
                # Add point and slot information
                sampled_data.update({
                    'point_id': point_id,
                    'point_date': point_date,
                    'slot': slot,
                    'sensor': 'S1',
                    'longitude': point_props['longitude'],
                    'latitude': point_props['latitude']
                })
                s1_results.append(sampled_data)
                
                # Show extraction info
                orbit_dir = sampled_data.get('s1_orbit_direction', 'Unknown')
                inc_angle = sampled_data.get('s1_angle', 'N/A')
                print(f"  {slot}: {orbit_dir} orbit, angle: {inc_angle}° - {sampled_data['acquisition_date']}")
        else:
            print(f"  {slot}: No image available")

print(f"\n✓ Sentinel-1 extraction complete: {len(s1_results)} records")

# Convert to DataFrame
s1_df = pd.DataFrame(s1_results)
if len(s1_df) > 0:
    print(f"✓ S1 DataFrame created: {s1_df.shape}")
    print(f"  Columns: {list(s1_df.columns)}")
    
    # Summary of orbit directions
    if 's1_orbit_direction' in s1_df.columns:
        orbit_summary = s1_df['s1_orbit_direction'].value_counts()
        print(f"\n=== ORBIT DIRECTION SUMMARY ===")
        for orbit, count in orbit_summary.items():
            print(f"  {orbit}: {count} acquisitions")
    
    # Summary of incidence angles
    if 's1_angle' in s1_df.columns:
        angles = s1_df['s1_angle'].dropna()
        if len(angles) > 0:
            print(f"\n=== INCIDENCE ANGLE SUMMARY ===")
            print(f"  Range: {angles.min():.1f}° to {angles.max():.1f}°")
            print(f"  Mean: {angles.mean():.1f}° ± {angles.std():.1f}°")
else:
    print("⚠ No Sentinel-1 data extracted")

=== EXTRACTING SENTINEL-1 BANDS ===
Including VV, VH polarizations, incidence angle, and orbit direction
Processing point 0 (1/23) - Date: 2025-06-29
  same:  orbit, angle: 36.10874557495117° - 2025-06-29
  same:  orbit, angle: 36.10874557495117° - 2025-06-29
  previous:  orbit, angle: 44.536293029785156° - 2025-06-28
  previous:  orbit, angle: 44.536293029785156° - 2025-06-28
  next:  orbit, angle: 35.952606201171875° - 2025-07-02
Processing point 1 (2/23) - Date: 2025-06-29
  next:  orbit, angle: 35.952606201171875° - 2025-07-02
Processing point 1 (2/23) - Date: 2025-06-29
  same:  orbit, angle: 36.11158752441406° - 2025-06-29
  same:  orbit, angle: 36.11158752441406° - 2025-06-29
  previous:  orbit, angle: 44.539031982421875° - 2025-06-28
  previous:  orbit, angle: 44.539031982421875° - 2025-06-28
  next:  orbit, angle: 35.941864013671875° - 2025-07-02
Processing point 2 (3/23) - Date: 2025-06-29
  next:  orbit, angle: 35.941864013671875° - 2025-07-02
Processing point 2 (3/23) - Dat

## 9. Merge and Structure Final Dataset

In [39]:
print("=== MERGING SENTINEL-1 AND SENTINEL-2 DATA ===")

# Combine S1 and S2 results
all_results = []

# Add S2 data
if len(s2_df) > 0:
    all_results.extend(s2_results)

# Add S1 data
if len(s1_df) > 0:
    all_results.extend(s1_results)

if len(all_results) > 0:
    # Create comprehensive DataFrame
    final_df = pd.DataFrame(all_results)
    
    print(f"✓ Combined dataset: {final_df.shape}")
    print(f"  Total extractions: {len(final_df)}")
    print(f"  Sensors: {final_df['sensor'].value_counts().to_dict()}")
    print(f"  Temporal slots: {final_df['slot'].value_counts().to_dict()}")
    
    # Reorder columns for better readability
    base_columns = ['point_id', 'point_date', 'slot', 'sensor', 'acquisition_date', 
                   'longitude', 'latitude', 'image_id']
    
    s2_columns = [col for col in final_df.columns if col.startswith('s2_')]
    s1_columns = [col for col in final_df.columns if col.startswith('s1_')]
    
    column_order = base_columns + s2_columns + s1_columns
    column_order = [col for col in column_order if col in final_df.columns]
    
    final_df = final_df[column_order]
    
    print(f"\n=== FINAL DATASET STRUCTURE ===")
    print(f"Columns ({len(final_df.columns)}):")
    for i, col in enumerate(final_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Summary statistics
    print(f"\n=== EXTRACTION SUMMARY ===")
    print("Data availability by sensor and slot:")
    summary = final_df.groupby(['sensor', 'slot']).size().unstack(fill_value=0)
    print(summary)
    
    # Show sample of final data
    print(f"\n=== SAMPLE DATA (First 5 rows) ===")
    display_cols = ['point_id', 'point_date', 'slot', 'sensor', 'acquisition_date']
    if 's2_B2' in final_df.columns:
        display_cols.append('s2_B2')
    if 's1_VV' in final_df.columns:
        display_cols.append('s1_VV')
    if 's1_orbit_direction' in final_df.columns:
        display_cols.append('s1_orbit_direction')
    
    sample_data = final_df[display_cols].head()
    print(sample_data.to_string(index=False))
    
else:
    print("⚠ No data to merge - check extraction steps")
    final_df = pd.DataFrame()

=== MERGING SENTINEL-1 AND SENTINEL-2 DATA ===
✓ Combined dataset: (76, 19)
  Total extractions: 76
  Sensors: {'S1': 56, 'S2': 20}
  Temporal slots: {'same': 33, 'previous': 23, 'next': 20}

=== FINAL DATASET STRUCTURE ===
Columns (19):
   1. point_id
   2. point_date
   3. slot
   4. sensor
   5. acquisition_date
   6. longitude
   7. latitude
   8. image_id
   9. s2_B2
  10. s2_B3
  11. s2_B4
  12. s2_B8
  13. s2_B11
  14. s2_B12
  15. s1_VV
  16. s1_VH
  17. s1_angle
  18. s1_orbit_direction
  19. s1_relative_orbit

=== EXTRACTION SUMMARY ===
Data availability by sensor and slot:
slot    next  previous  same
sensor                      
S1        10        23    23
S2        10         0    10

=== SAMPLE DATA (First 5 rows) ===
 point_id point_date slot sensor acquisition_date  s2_B2  s1_VV s1_orbit_direction
        0 2025-06-29 same     S2                   476.0    NaN                NaN
        0 2025-06-29 next     S2                   872.0    NaN                NaN
        

## 10. Export Results and Data Preview

In [40]:
if not final_df.empty:
    print("=== EXPORTING RESULTS ===")
    
    # Export main results to CSV
    output_filename = "multi_temporal_satellite_bands.csv"
    final_df.to_csv(output_filename, index=False)
    print(f"✓ Main dataset exported to: {output_filename}")
    
    # Create separate files for each sensor if needed
    if 'S2' in final_df['sensor'].values:
        s2_only = final_df[final_df['sensor'] == 'S2']
        s2_filename = "sentinel2_bands_multi_temporal.csv"
        s2_only.to_csv(s2_filename, index=False)
        print(f"✓ Sentinel-2 data exported to: {s2_filename}")
    
    if 'S1' in final_df['sensor'].values:
        s1_only = final_df[final_df['sensor'] == 'S1']
        s1_filename = "sentinel1_bands_multi_temporal.csv"
        s1_only.to_csv(s1_filename, index=False)
        print(f"✓ Sentinel-1 data exported to: {s1_filename}")
    
    # Create summary report
    print(f"\n=== EXTRACTION REPORT ===")
    print(f"Extraction completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total sampling points: {total_points}")
    print(f"Total extractions: {len(final_df)}")
    print(f"Configuration used:")
    for key, value in CONFIG.items():
        print(f"  {key}: {value}")
    
    # Data quality summary
    print(f"\n=== DATA QUALITY SUMMARY ===")
    
    # Check for missing temporal slots per point
    point_slot_counts = final_df.groupby(['point_id', 'sensor'])['slot'].nunique()
    complete_points = point_slot_counts[point_slot_counts == 3]  # same, previous, next
    
    if len(complete_points) > 0:
        print(f"Points with complete temporal coverage: {len(complete_points)}")
    
    # Cloud coverage impact (S2 only)
    if 'S2' in final_df['sensor'].values:
        s2_data = final_df[final_df['sensor'] == 'S2']
        s2_band_cols = [col for col in s2_data.columns if col.startswith('s2_B')]
        if s2_band_cols:
            null_percentage = (s2_data[s2_band_cols].isnull().sum() / len(s2_data) * 100).round(1)
            print(f"\nSentinel-2 cloud impact (% null values):")
            for band, pct in null_percentage.items():
                print(f"  {band}: {pct}%")
    
    # Orbit distribution (S1 only)
    if 'S1' in final_df['sensor'].values:
        s1_data = final_df[final_df['sensor'] == 'S1']
        if 's1_orbit_direction' in s1_data.columns:
            orbit_dist = s1_data['s1_orbit_direction'].value_counts()
            print(f"\nSentinel-1 orbit distribution:")
            for orbit, count in orbit_dist.items():
                print(f"  {orbit}: {count} acquisitions")
    
    print(f"\n=== NEXT STEPS ===")
    print("✓ Raw bands extracted and ready for analysis")
    print("✓ Use extracted bands to calculate spectral indices")
    print("✓ Perform temporal analysis (same vs previous vs next)")
    print("✓ Correlate with field soil moisture measurements")
    print("✓ Account for orbit direction and incidence angle in S1 analysis")
    print("✓ Handle cloud-affected S2 pixels appropriately")
    
else:
    print("⚠ No data to export - check extraction process")

print(f"\n{'='*60}")
print("MULTI-TEMPORAL BAND EXTRACTION COMPLETE")
print(f"{'='*60}")

=== EXPORTING RESULTS ===
✓ Main dataset exported to: multi_temporal_satellite_bands.csv
✓ Sentinel-2 data exported to: sentinel2_bands_multi_temporal.csv
✓ Sentinel-1 data exported to: sentinel1_bands_multi_temporal.csv

=== EXTRACTION REPORT ===
Extraction completed: 2025-09-28 22:02:00
Total sampling points: 23
Total extractions: 76
Configuration used:
  max_days_before: 10
  max_days_after: 10
  s2_bands: ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
  s1_bands: ['VV', 'VH', 'angle']
  s2_scale: 10
  s1_scale: 10
  aoi_buffer: 9e-05
  s1_speckle_filter_size: 30
  s1_angle_min: 30.64
  s1_angle_max: 45.24

=== DATA QUALITY SUMMARY ===
Points with complete temporal coverage: 10

Sentinel-2 cloud impact (% null values):
  s2_B2: 0.0%
  s2_B3: 0.0%
  s2_B4: 0.0%
  s2_B8: 0.0%
  s2_B11: 0.0%
  s2_B12: 0.0%

Sentinel-1 orbit distribution:
  : 56 acquisitions

=== NEXT STEPS ===
✓ Raw bands extracted and ready for analysis
✓ Use extracted bands to calculate spectral indices
✓ Perform temporal an

## Summary and Next Steps

This notebook has successfully implemented a comprehensive multi-temporal satellite band extraction methodology:

### ✅ **What Was Accomplished:**

1. **Multi-temporal Approach**: Extracted data from 3 time periods per point:
   - Same day as field measurement
   - Previous image (within 10-day window)
   - Next image (within 10-day window)

2. **Sentinel-2 Raw Bands**: Extracted B2, B3, B4, B8, B11, B12
   - Cloud pixels preserved as null values (no masking/replacement)
   - Maintains data integrity for cloud impact analysis

3. **Sentinel-1 with Metadata**: Extracted VV, VH, incidence angle
   - Orbit direction (ascending/descending) included
   - Acquisition metadata preserved

4. **Structured Output**: Tidy dataset ready for:
   - Spectral index calculation
   - Temporal analysis
   - Correlation with field measurements

### 🎯 **Key Advantages of This Approach:**

- **Date-Matched Extraction**: Ensures temporal consistency with field data
- **Multi-temporal Context**: Enables before/after comparisons
- **Cloud-Aware**: Preserves information about cloud contamination
- **SAR Metadata**: Includes important radar acquisition parameters
- **Flexible**: Easy to modify search windows and band selections

### 📊 **Ready for Analysis:**
The extracted data can now be used to:
1. Calculate spectral indices (NDVI, NDMI, etc.)
2. Analyze temporal patterns
3. Correlate with soil moisture measurements
4. Account for acquisition conditions (clouds, orbit direction)